# 국내 주식 데이터 수집

## 최근 영업일 기준 데이터 받기

In [3]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url) #해당 페이지 내용을 받아오기.
data_html = BeautifulSoup(data.content) #해당 페이지의 HTML 내용을 BeautifulSoup 객체로 만들기
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text #해당 태그의 데이터를 추출하며, text 메서드를 이용해 텍스트 데이터만을 추출

print(parse_day)

  |  2025.03.25


##### yyyy.mm.dd 형식 -> yyyymmdd 형태

In [5]:
import re

biz_day = re.findall('[0-9]+', parse_day) #정규 표현식을 이용해 숫자에 해당하는 부분만을 추출
biz_day = ''.join(biz_day) #숫자를 합치기

print(biz_day)

20250325


## 한국거래소의 업종분류 현황 및 개별지표 크롤링

주식 관련 데이터를 구하기 위해 가장 먼저 해야하는 일은 어떠한 종목들이 상장되어 있는가에 대한 정보를 구하는 것.

한국거래소에서 제공하는 업종분류 현황과 개별종목 지표 데이터를 이용하면 매우 간단하게 해당 정보를 수집이 가능

### 업종분류 현황 크롤링

#### KRX OTP 기반 데이터 크롤링 정리

<details>
<summary>OTP란?</summary>

***OTP (One-Time Password)*** 는 "서버에서 데이터를 요청하기 위한 임시 인증 키(token)"을 의미한다.

- 한국거래소(KRX)는 데이터를 직접 URL로 요청하지 못하게 막아둠.
- 대신 generate.cmd로 OTP를 발급받으면 데이터 요청 가능.
- 그 OTP를 download.cmd에 제출해야 실제 데이터(CSV)를 받을 수 있음.

</details>

---

<details>
<summary>전체 요청 흐름</summary>

1. **OTP 요청**  
   - `generate.cmd` URL에 데이터 요청 정보와 함께 POST 요청을 보냄  
   - 서버가 OTP 코드(문자열)를 응답

2. **OTP 제출**  
   - `download.cmd` URL에 OTP 코드를 POST로 제출  
   - 서버가 CSV 데이터 응답

</details>

---

<details>
<summary>필수 Header 설정</summary>

```python
headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
}
</details>

#### GET vs POST

<details>
<summary>GET과 POST의 차이점</summary>

### 📌 GET vs POST 정리표

| 항목 | GET | POST |
|------|-----|------|
| 데이터 위치 | URL에 포함됨 (쿼리스트링) | HTTP 메시지 본문(Body)에 포함 |
| 주 사용 목적 | 데이터 조회 (읽기) | 데이터 생성, 제출, 변경 |
| 주소창 표시 | 전송하는 데이터가 주소창에 보임 | 주소창에 보이지 않음 |
| 데이터 길이 제한 | 있음 (URL 최대 길이 제약) | 사실상 제한 없음 |
| 보안성 | 낮음 (주소 노출) | 상대적으로 높음 (Body에 숨김) |
| 캐싱 | 가능 (브라우저가 주소 저장) | 불가능 또는 제한적 |
| 즐겨찾기 | 가능 (URL 저장됨) | 어려움 |
| 사용 예시 | 검색, 조회 (ex: `?query=apple`) | 로그인, 글쓰기, 파일 업로드 등 |

---

In [11]:
#OTP 요청

import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
#2025년 1월 11일 기준 header에 'User-Agent'있어야 otp 발급이 진행됨
headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

F2LVFs8EecOpM4pyCPetddXxhNxcl503P/4TO3YGu9ARtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eXcFEkvAdh14xu0dti5fxBgtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [12]:
#코스피 데이터 다운

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers) #부여받은 OTP 코드를 해당 URL에 제출
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR') #

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,일반서비스,3795,-10,-0.26,171734220405
1,006840,AK홀딩스,KOSPI,기타금융,10100,-240,-2.32,133800366100
2,027410,BGF,KOSPI,기타금융,3460,0,0.00,331180096860
3,282330,BGF리테일,KOSPI,유통,111700,800,0.72,1930612300200
4,138930,BNK금융지주,KOSPI,기타금융,10780,-150,-1.37,3432174334820


In [13]:
#코스닥 데이터 다운

gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,의료·정밀기기,1840,0,0.00,97628633600
1,054620,APS,KOSDAQ,금융,6130,-10,-0.16,121951574730
2,265520,AP시스템,KOSDAQ,기계·장비,17950,-490,-2.66,274301506950
3,211270,AP위성,KOSDAQ,운송장비·부품,12220,-160,-1.29,184305754880
4,109960,AP헬스케어,KOSDAQ,유통,486,-15,-2.99,97583584734


In [14]:
# 코스피 데이터와 코스닥 데이터를 하나로 merge
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip() #공백제거
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,일반서비스,3795,-10,-0.26,171734220405,20250325
1,006840,AK홀딩스,KOSPI,기타금융,10100,-240,-2.32,133800366100,20250325
2,027410,BGF,KOSPI,기타금융,3460,0,0.00,331180096860,20250325
3,282330,BGF리테일,KOSPI,유통,111700,800,0.72,1930612300200,20250325
4,138930,BNK금융지주,KOSPI,기타금융,10780,-150,-1.37,3432174334820,20250325


### 개별종목 지표 크롤링

In [16]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)


krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,1840,0,0.00,54.0,34.07,NaN,NaN,998.0,1.84,0,0.00,20250325
1,095570,AJ네트웍스,3795,-10,-0.26,367.0,10.34,712.0,5.33,9326.0,0.41,270,7.11,20250325
2,006840,AK홀딩스,10100,-240,-2.32,2635.0,3.83,NaN,NaN,44339.0,0.23,200,1.98,20250325
3,054620,APS,6130,-10,-0.16,667.0,9.19,NaN,NaN,11683.0,0.52,0,0.00,20250325
4,265520,AP시스템,17950,-490,-2.66,3997.0,4.49,4152.0,4.32,21396.0,0.84,270,1.50,20250325


### 데이터 정리하기 

In [18]:
# 하나의 데이터에만 존재하는 종목 확인

diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['LG우', '제이알글로벌리츠', '씨엑스아이', '한국패러랠', '이노션', '이스트아시아홀딩스', 'NH올원리츠', '엑세스바이오', '삼성FN리츠', '잉글우드랩', '대신증권우 (락)', 'GRT', 'LG', '크리스탈신소재', 'SK리츠', '대신증권', 'ESR켄달스퀘어리츠', '대신증권2우B', '신한글로벌액티브리츠', '미래에셋맵스리츠', '애머릿지', '소마젠', '삼성화재', '케이탑리츠', '이지스레지던스리츠', 'KB스타리츠', '신한서부티엔디리츠', '삼성화재우', '롯데리츠', '코오롱티슈진', 'KB발해인프라', '한화리츠', '미래에셋글로벌리츠', '전진건설로봇 (락)', '맥쿼리인프라', '고스트스튜디오', '신한알파리츠', '전진건설로봇', '헝셩그룹', '삼성화재우 (락)', '한국ANKOR유전', '윙입푸드', 'LG우 (락)', '로스웰', '대신증권 (락)', '에이리츠', 'JTC', '마스턴프리미어리츠', '이노션 (락)', '네오이뮨텍', '맵스리얼티1', '대신증권우', 'LG (락)', 'SBI핀테크솔루션즈', '이지스밸류리츠', 'NH프라임리츠', '이리츠코크렙', '롯데이노베이트', '컬러레이', '스타에스엠리츠', '삼성화재 (락)', '디앤디플랫폼리츠', '롯데이노베이트 (락)', '오가닉티코스메틱', '글로벌에스엠', '코람코라이프인프라리츠', '엘브이엠씨홀딩스', '코람코더원리츠', '프레스티지바이오파마', '대신증권2우B (락)']


In [19]:
#두 데이터의 종목명 열을 세트 형태로 변경한 후 symmetric_difference() 메서드를 통해 하나의 데이터에만 있는 종목을 살펴보기
#일반적이지 않은 종목

kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,000020,동화약품,KOSPI,제약,6160,40,0.65,1.720579e+11,20250325,991.0,6.22,NaN,NaN,13413.0,0.46,180.0,2.92
1,000040,KR모터스,KOSPI,운송장비·부품,388,-3,-0.77,2.333155e+10,20250325,NaN,NaN,NaN,NaN,618.0,0.63,0.0,0.00
2,000050,경방,KOSPI,유통,6480,50,0.78,1.776509e+11,20250325,NaN,NaN,NaN,NaN,29623.0,0.22,125.0,1.93
3,000070,삼양홀딩스,KOSPI,기타금융,60300,-1400,-2.27,5.164255e+11,20250325,22269.0,2.71,NaN,NaN,257475.0,0.23,3500.0,5.80
4,000075,삼양홀딩스우,KOSPI,기타금융,59800,-400,-0.66,1.818267e+10,20250325,NaN,NaN,NaN,NaN,NaN,NaN,3550.0,5.94


#### 스팩(SPAC)

- Special Purpose Acquisition Company의 약자로 기업인수를 목적으로 하는 페이퍼컴퍼니를 뜻한다.
- 대부분 증권사 주관으로 설립되며, 스팩이 먼저 투자자들의 자금을 모아 주식 시장에 상장이 되고 나면, 그 이후에 괜찮은 비상장기업을 찾아 합병하는 방식으로 최종 기업 인수가 이루어진다.

In [21]:
# 스팩
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

['신한제10호스팩' '신영스팩8호' '유안타제10호스팩' '하나금융25호스팩' '에스케이증권제8호스팩' '엔에이치스팩24호'
 '엔에이치스팩25호' '삼성스팩7호' '엔에이치스팩26호' 'IBKS제20호스팩' '교보13호스팩' '엔에이치스팩27호'
 '유진스팩9호' '대신밸런스제14호스팩' 'IBKS제21호스팩' '미래에셋드림스팩1호' '유안타제11호스팩' '비엔케이제1호스팩'
 '신영스팩9호' '유안타제12호스팩' '미래에셋비전스팩2호' '하나26호스팩' '키움제8호스팩' '하나27호스팩' '삼성스팩8호'
 'IBKS제22호스팩' '미래에셋비전스팩3호' '유안타제13호스팩' '하이제8호스팩' '유안타제14호스팩' '엔에이치스팩29호'
 '상상인제4호스팩' '신한제11호스팩' '하나29호스팩' '하나28호스팩' 'KB제25호스팩' '한화플러스제4호스팩'
 '에스케이증권제9호스팩' 'DB금융스팩11호' '교보14호스팩' '대신밸런스제15호스팩' '대신밸런스제16호스팩'
 '에스케이증권제10호스팩' 'KB제26호스팩' '한국제12호스팩' '에이치엠씨제6호스팩' '한국제13호스팩' 'KB제27호스팩'
 '교보15호스팩' '엔에이치스팩30호' 'IBKS제23호스팩' '삼성스팩9호' '유진스팩10호' 'IBKS제24호스팩'
 '하나30호스팩' '하나31호스팩' '대신밸런스제17호스팩' '신영스팩10호' '에스케이증권제11호스팩' '에스케이증권제12호스팩'
 '유안타제15호스팩' '비엔케이제2호스팩' '에스케이증권제13호스팩' '유안타제16호스팩' '신한제12호스팩' '신한제13호스팩'
 '하나32호스팩' '하나33호스팩' 'KB제28호스팩' '에이치엠씨제7호스팩' '미래에셋비전스팩4호' '미래에셋비전스팩5호'
 '한국제14호스팩' 'DB금융스팩12호' '이베스트스팩6호' 'KB제29호스팩' '미래에셋비전스팩6호' '대신밸런스제18호스팩'
 '한국제15호스팩' '엔에이치스팩31호' '교보16호스팩' '미래에셋비전스팩7호' '대신밸런스제19호스팩' '하나34호스팩'
 '

#### 리츠(REITs)와 우선주 개념 정리

<details>
<summary>📌 리츠 (REITs: Real Estate Investment Trusts)</summary>

✅ 개념
- 부동산에 투자하는 회사를 통해 간접적으로 부동산에 투자하는 방식
- 상업용 부동산(오피스, 쇼핑몰, 물류센터 등)에서 발생하는 임대 수익 등을 배당 형태로 지급

✅ 주요 특징

| 항목       | 설명 |
|------------|------|
| 투자 대상   | 실물 부동산 (빌딩, 호텔, 물류창고 등) |
| 수익원     | 임대 수익 + 부동산 가치 상승 |
| 배당       | 수익의 90% 이상 배당해야 세제 혜택 |
| 유동성     | 상장 리츠는 주식처럼 자유롭게 거래 가능 |

✅ 장점
- 소액으로 부동산 투자 가능
- 주기적인 배당 수익
- 주식처럼 매매 가능 → 유동성 확보

✅ 단점
- 금리 상승 시 리츠 가격 하락 가능
- 부동산 경기 민감
- 배당에 세금 부과

</details>

---

<details>
<summary>📌 우선주 (Preferred Stock)</summary>

✅ 개념
- 보통주보다 **배당 우선권이 있는 주식**
- 일반적으로 **의결권은 없음**

✅ 주요 특징

| 항목       | 설명 |
|------------|------|
| 배당 순위   | 보통주보다 우선 배당 |
| 의결권     | 없음 (대부분의 경우) |
| 안정성     | 정해진 배당률, 보통주보다 안정적 |
| 유동성     | 보통주보다 낮을 수 있음 |

✅ 장점
- 안정적인 배당 수익
- 고배당 매력 (주가가 낮게 형성된 경우)
- 일부 우선주는 보통주로 전환 가능

✅ 단점
- 의결권 없음 → 기업 경영 영향력 없음
- 주가 상승 제한적
- 거래량 부족 시 유동성 문제

</details>

---

<details>
<summary>📊 리츠 vs 우선주 비교</summary>

| 항목       | 리츠 | 우선주 |
|------------|------|--------|
| 투자 대상   | 부동산 | 기업 주식 |
| 수익원     | 임대 수익 + 자산 가치 상승 | 배당 수익 |
| 배당 안정성 | 부동산 경기 영향 큼 | 기업 수익성과 연동 |
| 의결권     | 없음 | 없음 |
| 거래 방식   | 상장 리츠는 주식처럼 거래 | 주식처럼 거래 |
| 주요 리스크 | 부동산 시장, 금리 민감 | 기업의 실적 및 재무 상태 |

</details>

---

<details>
<summary>📝 투자 시 고려할 점</summary>

- 리츠는 **금리 변화**, **부동산 경기**에 민감함
- 우선주는 **배당 매력은 있지만 유동성이 낮을 수 있음**
- 본인의 투자 목적(배당 수익 vs 성장성)에 따라 선택 필요

</details>


In [23]:
# 우선주
# 국내 종목 중 종목코드 끝이 0이 아닌 종목은 우선주에 해당
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['삼양홀딩스우' '하이트진로2우B' '유한양행우' '하이트진로홀딩스우' '두산우' '두산2우B' 'DL우' '유유제약1우'
 '유유제약2우B' '노루홀딩스우' '흥국화재우' '현대건설우' '삼성화재우' '삼성화재우 (락)' '한화우' '한화3우B'
 'CJ우' 'CJ4우(전환)' 'JW중외제약우' 'JW중외제약2우B' '부국증권우' 'BYC우' 'SK증권우' '동양우'
 '동양2우B' '대상우' '한양증권우' '대한제당우' '코오롱우' '넥센타이어1우B' '진흥기업우B' '진흥기업2우B'
 '아모레G우' '아모레G3우(전환)' '금호건설우' '코오롱글로벌우' '유화증권우' '유안타증권우' '대한항공우' '한화투자증권우'
 '대신증권우' '대신증권우 (락)' '대신증권2우B' '대신증권2우B (락)' 'LG우' 'LG우 (락)' '남양유업우'
 '태양금속우' 'NPC우' '세방우' '서울식품우' '깨끗한나라우' '덕성우' '성신양회우' '롯데지주우' '녹십자홀딩스2우'
 '롯데칠성우' '현대차우' '현대차2우B' '현대차3우B' '넥센우' '크라운해태홀딩스우' '삼성전자우' 'NH투자증권우'
 '동부건설우' 'SK디스커버리우' '대원전선우' '삼성SDI우' '미래에셋증권우' '미래에셋증권2우B' '일양약품우' '코리아써우'
 '코리아써키트2우B' '대덕1우' '남선알미우' '호텔신라우' '삼성전기우' '태영건설우' '한화솔루션우' 'S-Oil우'
 'CJ씨푸드1우' '금호석유우' '계양전기우' '금강공업우' '동원시스템즈우' '성문전자우' '신풍제약우' '대교우B'
 '대호특수강우' '삼성물산우B' '소프트센우' 'SK우' '해성산업1우' 'LG생활건강우' 'LG화학우' 'LG전자우'
 '한국금융지주우' 'GS우' '대상홀딩스우' '노루페인트우' '아모레퍼시픽우' 'SK이노베이션우' 'CJ제일제당 우'
 'LX하우시스우' '코오롱인더우' '삼양사우' '한진칼우' '크라운제과우' 'SK케미칼우' '두산퓨얼셀1우' '두산퓨얼셀2우B'
 '솔루스첨단소재1우' '솔루

In [24]:
# 리츠
#리츠 종목은 종목명이 '리츠'로 끝남
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

['에이리츠' '케이탑리츠' '스타에스엠리츠' '신한알파리츠' '롯데리츠' '이지스밸류리츠' 'NH프라임리츠' '제이알글로벌리츠'
 '이지스레지던스리츠' '코람코라이프인프라리츠' '미래에셋맵스리츠' '마스턴프리미어리츠' 'ESR켄달스퀘어리츠' '디앤디플랫폼리츠'
 'SK리츠' '미래에셋글로벌리츠' 'NH올원리츠' '신한서부티엔디리츠' '코람코더원리츠' 'KB스타리츠' '삼성FN리츠'
 '한화리츠' '신한글로벌액티브리츠']


In [25]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,000020,동화약품,KOSPI,6160,172057855200.0,2025-03-25,991.0,None,13413.0,180.0,보통주
1,000040,KR모터스,KOSPI,388,23331552784.0,2025-03-25,None,None,618.0,0.0,보통주
2,000050,경방,KOSPI,6480,177650949600.0,2025-03-25,None,None,29623.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,60300,516425541300.0,2025-03-25,22269.0,None,257475.0,3500.0,보통주
4,000075,삼양홀딩스우,KOSPI,59800,18182668400.0,2025-03-25,None,None,None,3550.0,우선주


In [ ]:
import pymysql

con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)

mycursor = con.cursor()
query = f"""
    insert into kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
    values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    종목명=new.종목명,시장구분=new.시장구분,종가=new.종가,시가총액=new.시가총액,EPS=new.EPS,선행EPS=new.선행EPS,
    BPS=new.BPS,주당배당금=new.주당배당금,종목구분 = new.종목구분;
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

## WICS 기준 섹터정보 크롤링

<details>
<summary>WICS 기준 섹터 정보란?</summary>

### WICS란?
- **WICS**는 *Wise Industry Classification Standard*의 약자로, **FnGuide**에서 제공하는 **국내 주식 섹터 분류 체계**이다.
- WICS는 GICS (Global Industry Classification Standard)와 유사한 구조로, **산업을 여러 단계(Level 1~4)** 로 구분해 체계적으로 분류한다.

</details>

---

<details>
<summary>GICS vs WICS</summary>

### GICS (Global Industry Classification Standard)
- MSCI와 S&P가 공동 개발한 글로벌 산업분류 체계
- 전 세계적으로 가장 널리 사용되며, 금융 데이터/리서치 업체에서도 표준처럼 활용

### WICS
- GICS를 참고해 만든 **국내 산업분류 시스템**
- FnGuide가 자체적으로 관리하고 운영
- 국내 종목 중심으로 세분화되어 있으며, GICS와의 구조가 비슷해 비교·분석에 용이

| 구분 | GICS | WICS |
|------|------|------|
| 제공 주체 | MSCI & S&P | FnGuide |
| 적용 범위 | 글로벌 | 국내 중심 |
| 사용 제한 | 유료, 저작권 있음 | 공개적으로 접근 가능 (웹 기준) |

</details>

---

<details>
<summary>왜 WICS를 사용하는가?</summary>

- GICS는 전 세계적으로 널리 쓰이지만, **한국거래소에서 제공하는 GICS 정보는 지적재산권 제한**이 있다.
- GICS 정보는 유료이거나 사용에 제약이 있음
- **WICS는 웹에서 공개되어 있고, 크롤링 등으로 접근 가능**하여 활용도가 높음
- 특히 **섹터 기반 종목 분석, 섹터 ETF 구성 파악 등**에 유용

</details>

---

<details>
<summary>WICS 섹터 구조 (예시)</summary>

WICS는 **총 4단계 분류 체계**로 구성됨:

1. 대분류 (Sector)
2. 중분류 (Industry Group)
3. 소분류 (Industry)
4. 세부분류 (Sub-Industry)

예시:

- 정보기술 (IT)
  - 소프트웨어
    - 응용 소프트웨어
      - 게임 소프트웨어

이런 구조를 통해 **기업의 주요 산업군을 보다 세분화**해서 분류할 수 있다.

</details>

---

<details>
<summary>WICS 데이터 크롤링 활용</summary>

- FnGuide 또는 금융 정보 제공 웹사이트에서 WICS 기준 섹터/업종 분류 정보를 크롤링 가능
- 크롤링 후 종목 코드, 종목명과 함께 **WICS 섹터 정보**를 병합하면
  - 섹터별 분석
  - 동일 업종 비교
  - 포트폴리오 구성 등 다양한 데이터 분석에 활용할 수 있다

</details>

---

<details>
<summary>WICS 구성종목 데이터 구조 이해</summary>

### 1. 데이터 위치
- 웹사이트: http://www.wiseindex.com/Index
- 경로: 좌측 메뉴 → WISE SECTOR INDEX → WICS → (예: 에너지) → Components 탭 클릭
- **Components 탭**에서는 선택한 섹터의 **구성 종목 리스트**를 제공

### 2. 데이터 요청 URL 분석
- 구성종목 데이터는 다음과 같은 주소를 통해 받아온다: http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20220419&sec_cd=G10


- URL 파라미터 설명:

| 파라미터 | 의미 |
|----------|------|
| `ceil_yn=0` | 실링 여부 (0 = 비실링) |
| `dt=20220419` | 조회 일자 (예: 2022년 4월 19일) |
| `sec_cd=G10` | 섹터 코드 (예: G10 = 에너지) |

### 3. 응답 형식 (JSON)
- 응답은 **JSON 형식**으로 제공됨
- 일반적인 HTML과 달리, **구조화된 데이터 형식**으로 바로 파싱 가능
- Python에서는 `json.loads()` 또는 `response.json()`을 사용하여 손쉽게 처리 가능

### 4. 요약
- WICS 구성종목은 HTML이 아닌 **JSON API**로 따로 요청 가능
- 크롤링 시 URL 구성만 잘 파악하면 쉽게 자동화 가능
- JSON → DataFrame 변환도 간단함

</details>


In [29]:
import json
import requests as rq
import pandas as pd

url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
data = rq.get(url).json()

type(data)

dict

In [30]:
# list 부분의 데이터를 데이터프레임 형태로 변경
data_pd = pd.json_normalize(data['list'])

data_pd.head()

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,29712662,096770,SK이노베이션,10496313,35.33,35.33,1.0,G10,에너지,1,3,84579475
1,G10,WICS 에너지,29712662,034730,SK,4629153,15.58,50.91,1.0,G10,에너지,2,3,33351243
2,G10,WICS 에너지,29712662,267250,HD현대,3308862,11.14,62.04,1.0,G10,에너지,3,3,44236128
3,G10,WICS 에너지,29712662,010950,S-Oil,2407696,8.10,70.15,1.0,G10,에너지,4,3,41655633
4,G10,WICS 에너지,29712662,009830,한화솔루션,2116031,7.12,77.27,1.0,G10,에너지,5,3,108292298


In [31]:
#URL의 sec_cd=에 해당하는 부분만 변경하면 모든 섹터의 구성종목을 얻을 수 있음 -> for문 사용

import time
import json
import requests as rq
import pandas as pd
from tqdm import tqdm

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45'
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''    
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(2)

kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

100%|███████████████████████████████████████████| 10/10 [00:47<00:00,  4.71s/it]


In [ ]:
import pymysql

con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)

mycursor = con.cursor()
query = f"""
    insert into kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
    values (%s,%s,%s,%s,%s) as new
    on duplicate key update
    IDX_CD = new.IDX_CD, CMP_KOR = new.CMP_KOR, SEC_NM_KOR = new.SEC_NM_KOR
"""

args = kor_sector.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

## 수정주가 크롤링

<details>
<summary>수정주가란?</summary>

### ✅ 수정주가란?
- **액면분할, 무상증자, 배당락 등 기업의 이벤트로 인한 주가 변동을 반영한 조정 주가**를 의미한다.
- 실제 투자자의 **자산가치 변화 없이 발생한 가격 변화**를 보정하여 **과거부터 현재까지 주가의 연속성을 유지**하는 방식이다.

</details>

---

<details>
<summary>왜 수정주가가 필요한가?</summary>

### 예시: 삼성전자 액면분할
- 2018년 5월, 삼성전자는 1주를 50주로 나누는 **액면분할**을 실시
- 주가는 **265만 원 → 5만 3천 원**으로 급락한 듯 보이지만 실제로는 **보유 주식 수가 50배로 증가**했기 때문에 자산은 동일하다

### 문제점
- 일반 주가 데이터를 그대로 사용할 경우 → 과거 대비 -98% 수익률처럼 **왜곡된 계산 결과** 발생 가능

### 해결책
- 액면분할 이전의 모든 주가를 50으로 나누어 조정
- 이처럼 이벤트를 반영해 **연속적 주가 흐름을 유지한 값이 수정주가**

</details>

---

<details>
<summary>수정주가가 사용되는 이유</summary>

### 백테스트, 퀀트 투자에 필수
- 주가의 실제 흐름을 반영하지 않으면 **전략 수익률이 왜곡**
- 종목 선정, 이동평균선 계산, 수익률 비교 등에서 **정확성 확보**

### 실제 사용 예시
- 백테스트에서 2017~2024년 삼성전자 수익률을 계산할 때
- 분할, 배당 이벤트를 모두 반영해 **투자 성과를 현실적으로 측정**

</details>

---

<details>
<summary>수정주가 데이터는 어디서 구하나?</summary>

- 네이버 금융(https://finance.naver.com)에서 각 종목의 수정주가를 제공
- `day` 탭의 가격 데이터는 이미 수정주가 기준으로 반영되어 있음
- Python에서는 `requests`, `BeautifulSoup`, `pandas` 등을 활용해 **자동으로 수정주가 데이터를 크롤링**할 수 있음

</details>

### 개별종목 주가 크롤링

In [ ]:
import pymysql
import pandas as pd

# MySQL 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)

# 쿼리 실행
query = """
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
"""

# 결과 가져오기
ticker_list = pd.read_sql(query, conn)

# 연결 종료
conn.close()

# 결과 확인
ticker_list.head()


/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/332773820.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql(query, conn)


,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,000020,동화약품,KOSPI,6160.0,1.720580e+11,2025-03-25,991.0,NaN,13413.0,180.0,보통주
1,000040,KR모터스,KOSPI,388.0,2.333160e+10,2025-03-25,NaN,NaN,618.0,0.0,보통주
2,000050,경방,KOSPI,6480.0,1.776510e+11,2025-03-25,NaN,NaN,29623.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,60300.0,5.164260e+11,2025-03-25,22269.0,NaN,257475.0,3500.0,보통주
4,000080,하이트진로,KOSPI,19770.0,1.386540e+12,2025-03-25,512.0,1861.0,15694.0,950.0,보통주


In [37]:
#주가 데이터 페이지 크롤링
#날짜 및 주가, 거래량, 외국인소진율 데이터가 추출

from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = ticker_list['종목코드'][i]  #종목의 티커를 선택
fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d") #
to = (date.today()).strftime("%Y%m%d")

url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
&startTime={fr}&endTime={to}&timeframe=day'''

data = rq.get(url).content  #페이지의 데이터를 불러온 후, content 부분을 추출
data_price = pd.read_csv(BytesIO(data)) #바이너리스트림 형태 -> 데이터를 읽어오기

data_price.head()

,[['날짜','시가','고가','저가','종가','거래량','외국인소진율'],Unnamed: 7
0,"[""20200327""",6300.0,6300.0,5900.0,6130.0,193283.0,5.75],NaN
1,"[""20200330""",6130.0,6350.0,6000.0,6350.0,190170.0,5.84],NaN
2,"[""20200331""",6400.0,6650.0,6360.0,6550.0,143491.0,5.83],NaN
3,"[""20200401""",6610.0,6780.0,6500.0,6580.0,163650.0,5.84],NaN
4,"[""20200402""",6520.0,6680.0,6420.0,6610.0,146475.0,5.89],NaN


In [38]:
#클렌징 작업
import re

price = data_price.iloc[:, 0:6]
price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량'] #인덱서를 통해 날짜와 가격(시가, 고가, 저가, 종가), 거래량에 해당하는 데이터만을 선택
price = price.dropna() #  NA 데이터를 삭제
price['날짜'] = price['날짜'].str.extract('(\d+)') #정규 표현식을 이용해 날짜 열에서 숫자만을 추출
price['날짜'] = pd.to_datetime(price['날짜']) #'날짜'열을 datetime 형태로 변경
price['종목코드'] = ticker #'종목코드'열에 티커를 입력

price.head()

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/961405857.py:7: SyntaxWarning: invalid escape sequence '\d'
  price['날짜'] = price['날짜'].str.extract('(\d+)') #정규 표현식을 이용해 날짜 열에서 숫자만을 추출


,날짜,시가,고가,저가,종가,거래량,종목코드
0,2020-03-27,6300.0,6300.0,5900.0,6130.0,193283.0,000020
1,2020-03-30,6130.0,6350.0,6000.0,6350.0,190170.0,000020
2,2020-03-31,6400.0,6650.0,6360.0,6550.0,143491.0,000020
3,2020-04-01,6610.0,6780.0,6500.0,6580.0,163650.0,000020
4,2020-04-02,6520.0,6680.0,6420.0,6610.0,146475.0,000020


### 전 종목 주가 크롤링

In [ ]:
# 전종목 주가 DB에 저장

# 패키지 불러오기
import pymysql
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import requests as rq
import time
from tqdm import tqdm
from io import BytesIO

# MySQL 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
""", con=con)

# DB 저장 쿼리
query = """
    INSERT INTO kor_price (날짜, 시가, 고가, 저가, 종가, 거래량, 종목코드)
    VALUES (%s,%s,%s,%s,%s,%s,%s) AS new
    ON DUPLICATE KEY UPDATE
        시가 = new.시가,
        고가 = new.고가,
        저가 = new.저가,
        종가 = new.종가,
        거래량 = new.거래량;
"""

# 오류 발생시 저장할 리스트 생성
error_list = []

# 전종목 주가 다운로드 및 저장
for i in tqdm(range(len(ticker_list))):

    ticker = ticker_list['종목코드'][i]
    fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
    to = date.today().strftime("%Y%m%d")

    try:
        # 네이버 주가 URL 생성
        url = f"https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day"

        # 데이터 다운로드 및 파싱
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data), header=None)

        # 데이터 클렌징
        price = data_price.iloc[1:, 0:6]  # 첫 줄 제거
        price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
        price = price.dropna()
        price['날짜'] = price['날짜'].astype(str).str.extract(r'(\d+)')  # 날짜 숫자만 추출
        price['날짜'] = pd.to_datetime(price['날짜'], format='%Y%m%d')
        price['종목코드'] = ticker

        # DB 저장
        args = price.values.tolist()
        mycursor.executemany(query, args)
        con.commit()

    except Exception as e:
        print(f"[오류] {ticker}: {e}")
        error_list.append(ticker)

    # 너무 빠른 요청 방지
    time.sleep(2)

# 연결 종료
con.close()

# 오류 종목 출력
print("저장 실패 종목 수:", len(error_list))
print("오류 리스트:", error_list)

/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/2471700190.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql("""
100%|█████████████████████████████████████| 2492/2492 [2:11:59<00:00,  3.18s/it]

저장 실패 종목 수: 0
오류 리스트: []


### 재무제표 크롤링

In [ ]:
import pymysql
import pandas as pd

# 1. DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)

# 2. 종목코드 불러오기 (보통주만, 최신 기준일 기준)
query = """
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query, con=conn)

# 연결 종료
conn.close()

# 3. 티커 선택
i = 0  # 원하는 index로 바꿔도 됨
ticker = ticker_list['종목코드'][i]

# 4. FnGuide 재무제표 페이지에서 HTML 테이블 수집
url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
data = pd.read_html(url, displayed_only=False)

# 5. 각 테이블의 앞부분 출력
for idx, item in enumerate(data):
    print(f"\n[테이블 {idx}]")
    print(item.head(3))


/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/1503583686.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql(query, con=conn)



[테이블 0]
  IFRS(연결)  2021/12  2022/12  2023/12  2024/12    전년동기 전년동기(%)
0      매출액   2930.0   3404.0   3611.0   4649.0  3611.0    28.7
1     매출원가   1437.0   1594.0   1707.0   2506.0  1707.0    46.8
2    매출총이익   1493.0   1810.0   1904.0   2143.0  1904.0    12.5

[테이블 1]
  IFRS(연결)  2024/03  2024/06  2024/09  2024/12   전년동기 전년동기(%)
0      매출액   1189.0   1152.0   1102.0   1206.0  842.0    43.2
1     매출원가    639.0    612.0    597.0    658.0  377.0    74.7
2    매출총이익    549.0    540.0    505.0    549.0  466.0    17.8

[테이블 2]
             IFRS(연결)  2021/12  2022/12  2023/12  2024/12
0                  자산   4478.0   4611.0   5585.0   6196.0
1  유동자산계산에 참여한 계정 펼치기   2202.0   2275.0   2377.0   2378.0
2                재고자산    362.0    468.0    707.0    893.0

[테이블 3]
             IFRS(연결)  2024/03  2024/06  2024/09  2024/12
0                  자산   5581.0   5587.0   5871.0   6196.0
1  유동자산계산에 참여한 계정 펼치기   2337.0   2266.0   2182.0   2378.0
2                재고자산    704.0    732.0    775.0    893.0


In [43]:
#연간 기준 포괄손익계산서, 재무상태표, 현금흐름표의 열 이름
print(data[0].columns.tolist(), '\n',
      data[2].columns.tolist(), '\n',
      data[4].columns.tolist()
     )

['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/12', '전년동기', '전년동기(%)'] 
 ['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/12'] 
 ['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/12']


In [44]:
#포괄손익계산서 테이블에는 '전년동기', '전년동기(%)' 열이 있으며, 이는 필요하지 않은 내용이므로 삭제
data_fs_y = pd.concat(
    [data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]]) #포괄손익계산서 중 '전년동기'라는 글자가 들어간 열을 제외한 데이터를 선택/ 포괄손익계산서, 재무상태표, 현금흐름표 세개 테이블을 하나
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

data_fs_y.head()

,계정,2021/12,2022/12,2023/12,2024/12
0,매출액,2930.0,3404.0,3611.0,4649.0
1,매출원가,1437.0,1594.0,1707.0,2506.0
2,매출총이익,1493.0,1810.0,1904.0,2143.0
3,판매비와관리비계산에 참여한 계정 펼치기,1269.0,1511.0,1716.0,2008.0
4,인건비,468.0,489.0,521.0,606.0


In [45]:
import requests as rq
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content, 'html.parser') #페이지의 데이터를 불러온 후, content 부분을 BeautifulSoup 객체로

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

print(fiscal_data_text)

['12']


In [46]:
#연간 재무제표에 해당하는 열만 선택
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y.head()

,계정,2021/12,2022/12,2023/12,2024/12
0,매출액,2930.0,3404.0,3611.0,4649.0
1,매출원가,1437.0,1594.0,1707.0,2506.0
2,매출총이익,1493.0,1810.0,1904.0,2143.0
3,판매비와관리비계산에 참여한 계정 펼치기,1269.0,1511.0,1716.0,2008.0
4,인건비,468.0,489.0,521.0,606.0


In [47]:
#열 이름이 '계정', 그리고 재무제표의 월이 결산월과 같은 부분만 선택
data_fs_y[data_fs_y.loc[:, ~data_fs_y.columns.isin(['계정'])].isna().all(
    axis=1)].head()

,계정,2021/12,2022/12,2023/12,2024/12
10,기타원가성비용,NaN,NaN,NaN,NaN
18,대손충당금환입액,NaN,NaN,NaN,NaN
19,매출채권처분이익,NaN,NaN,NaN,NaN
20,당기손익-공정가치측정 금융자산관련이익,NaN,NaN,NaN,NaN
23,금융자산손상차손환입,NaN,NaN,NaN,NaN


In [48]:
#먼저 재무제표 값 중에서 모든 연도의 데이터가 NaN인 항목존재 -> 재무제표 계정은 있으나 해당 종목들은 데이터가 없는 것들이므로 삭제
data_fs_y['계정'].value_counts(ascending=False).head()

계정
기타          4
배당금수익       3
파생상품이익      3
이자수익        3
법인세납부(-)    3
Name: count, dtype: int64

In [49]:
def clean_fs(df, ticker, frequency):
    """
    재무제표 원시 데이터를 정제하여 분석 및 DB 저장이 가능한 형태로 변환하는 함수

    Parameters:
    - df (DataFrame): 원본 재무제표 테이블
    - ticker (str): 종목 코드
    - frequency (str): 'y' 또는 'q' (연간 또는 분기)

    Returns:
    - DataFrame: 정제된 재무제표 데이터
    """

    # 연도 값들이 전부 NaN인 항목 제거
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]

    # 중복된 계정명 제거 (첫 번째 항목만 사용)
    df = df.drop_duplicates(['계정'], keep='first')

    # wide → long 형태로 변환
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')

    # 값이 없는 항목 제거
    df = df[~pd.isnull(df['값'])]

    # '계산에 참여한 계정 펼치기' 같은 텍스트 제거
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)

    # 기준일 날짜 변환 (슬래시 → 하이픈 처리 후 변환)
    df['기준일'] = pd.to_datetime(df['기준일'].str.replace('/', '-'), format='%Y-%m') + pd.tseries.offsets.MonthEnd()

    # 종목코드 및 공시구분 추가
    df['종목코드'] = ticker
    df['공시구분'] = frequency

    return df

In [50]:
#연간 재무제표 항목에 위 함수를 적용
data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
data_fs_y_clean.head()

,계정,기준일,값,종목코드,공시구분
0,매출액,2021-12-31,2930.0,000020,y
1,매출원가,2021-12-31,1437.0,000020,y
2,매출총이익,2021-12-31,1493.0,000020,y
3,판매비와관리비,2021-12-31,1269.0,000020,y
4,인건비,2021-12-31,468.0,000020,y


In [51]:
# 분기 데이터

data_fs_q = pd.concat(
    [data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]])
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

data_fs_q_clean.head()

,계정,기준일,값,종목코드,공시구분
0,매출액,2024-03-31,1189.0,000020,q
1,매출원가,2024-03-31,639.0,000020,q
2,매출총이익,2024-03-31,549.0,000020,q
3,판매비와관리비,2024-03-31,484.0,000020,q
4,인건비,2024-03-31,153.0,000020,q


In [52]:
data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])

### 전종목 재무제표 크롤링

In [ ]:
# 전종목 재무제표 DB에 저장

# 패키지 불러오기
import pymysql
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

# DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
cursor = conn.cursor()

# 티커리스트 불러오기 (보통주 기준 최신일자)
query_ticker = """
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query_ticker, con=conn)

# DB 저장 쿼리
query_insert = """
    INSERT INTO kor_fs (계정, 기준일, 값, 종목코드, 공시구분)
    VALUES (%s,%s,%s,%s,%s) AS new
    ON DUPLICATE KEY UPDATE
    값 = new.값
"""

# 오류 발생시 저장할 리스트 생성
error_list = []

# 재무제표 클렌징 함수
def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'].str.replace('/', '-'), format='%Y-%m') + pd.tseries.offsets.MonthEnd()
    df['종목코드'] = ticker
    df['공시구분'] = frequency
    return df

# 전종목 루프
for i in tqdm(range(len(ticker_list))):
    ticker = ticker_list['종목코드'][i]

    try:
        url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        data = pd.read_html(url, displayed_only=False)

        # 연간 데이터
        data_fs_y = pd.concat([
            data[0].iloc[:, ~data[0].columns.str.contains('전년동기')],
            data[2],
            data[4]
        ])
        data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

        # 결산년 추출
        page_data = rq.get(url)
        soup = BeautifulSoup(page_data.content, 'html.parser')
        fiscal_data = soup.select('div.corp_group1 > h2')
        fiscal_text = re.findall('[0-9]+', fiscal_data[1].text)
        data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') | (data_fs_y.columns.str[-2:].isin(fiscal_text))]

        # 연간 데이터 클렌징
        data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')

        # 분기 데이터
        data_fs_q = pd.concat([
            data[1].iloc[:, ~data[1].columns.str.contains('전년동기')],
            data[3],
            data[5]
        ])
        data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
        data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

        # 통합 및 저장
        data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
        args = data_fs_bind.values.tolist()
        cursor.executemany(query_insert, args)
        conn.commit()

    except Exception as e:
        print(f"[오류] {ticker}: {e}")
        error_list.append(ticker)

    time.sleep(2)

# DB 연결 종료
conn.close()

# 에러 종목 출력
print("오류 종목 수:", len(error_list))
print("오류 목록:", error_list)


/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/2447882683.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql(query_ticker, con=conn)
100%|█████████████████████████████████████| 2492/2492 [3:58:03<00:00,  5.73s/it]

오류 종목 수: 0
오류 목록: []


### 가치지표 계산

<details>
<summary><strong>📌 가치지표란?</strong></summary>

가치지표(Value Indicators)는 기업의 가치를 평가하기 위해 사용하는 주요 지표.  
주가와 재무제표 데이터를 기반으로 계산되며, 대표적인 지표는 다음과 같음:

| 지표 | 설명 | 필요한 재무제표 데이터 |  
| --- | --- | --- |  
| **PER** | Price to Earnings Ratio (주가수익비율) | Earnings (순이익) |  
| **PBR** | Price to Book Ratio (주가순자산비율) | Book Value (순자산) |  
| **PCR** | Price to Cash Flow Ratio (주가현금흐름비율) | Cash Flow (영업활동현금흐름) |  
| **PSR** | Price to Sales Ratio (주가매출액비율) | Sales (매출액) |  
| **DY** | Dividend Yield (배당수익률) | Dividend (배당) |

</details>

---

<details>
<summary><strong>⏱ TTM (Trailing Twelve Months) 방식이란?</strong></summary>

- 연간 재무제표 기준 가치지표는 **다음 발표까지 1년 이상 기다려야** 하는 단점이 있음.
- 반면 **분기 재무제표는 3개월마다 발표**되므로, 최근 4개 분기 데이터를 활용한 **TTM 방식**이 일반적으로 사용됨.

📌 **TTM 방식의 장점**
- 최신 데이터 반영 가능
- 시장 반응 속도 향상
- 실시간 투자 판단에 유리

</details>




In [ ]:
# 📦 패키지 불러오기
import pymysql
import pandas as pd

# ✅ pymysql로 DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
cursor = conn.cursor()

# 📌 티커 리스트 불러오기 (보통주 기준 최신일자)
query_ticker = """
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query_ticker, con=conn)

# 📌 삼성전자 분기 재무제표 불러오기
query_fs = """
    SELECT * FROM kor_fs
    WHERE 공시구분 = 'q'
      AND 종목코드 = '005930'
      AND 계정 IN ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
"""
sample_fs = pd.read_sql(query_fs, con=conn)

# 🔚 연결 종료
cursor.close()
conn.close()

/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/2441499197.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql(query_ticker, con=conn)
/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/2441499197.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample_fs = pd.read_sql(query_fs, con=conn)


In [100]:
#재무제표 데이터를 종목코드, 계정, 기준일 순으로 정렬
sample_fs = sample_fs.sort_values(['종목코드', '계정', '기준일'])

sample_fs.head()

,계정,기준일,값,종목코드,공시구분,ttm
0,당기순이익,2024-03-31,67547.0,005930,q,NaN
1,당기순이익,2024-06-30,98413.0,005930,q,NaN
2,당기순이익,2024-09-30,101009.0,005930,q,NaN
3,당기순이익,2024-12-31,77544.0,005930,q,344513.0
4,매출액,2024-03-31,719156.0,005930,q,NaN


In [104]:
#4개 분기 데이터를 통해 TTM 값을 계산하며, 12개월치 데이터가 없을 경우는 계산을 하지 않기.
sample_fs['ttm'] = sample_fs.groupby(
    ['종목코드', '계정'], as_index=False)['값'].rolling(window=4,
                                                 min_periods=4).sum()['값']

sample_fs

,계정,기준일,값,종목코드,공시구분,ttm
0,당기순이익,2024-03-31,67547.0,005930,q,NaN
1,당기순이익,2024-06-30,98413.0,005930,q,NaN
2,당기순이익,2024-09-30,101009.0,005930,q,NaN
3,당기순이익,2024-12-31,77544.0,005930,q,344513.0
4,매출액,2024-03-31,719156.0,005930,q,NaN
5,매출액,2024-06-30,740683.0,005930,q,NaN
6,매출액,2024-09-30,790987.0,005930,q,NaN
7,매출액,2024-12-31,757883.0,005930,q,3008709.0
8,영업활동으로인한현금흐름,2024-03-31,118663.0,005930,q,NaN
9,영업활동으로인한현금흐름,2024-06-30,168954.0,005930,q,NaN


In [108]:
#1. '자본' 항목은 재무상태표에 해당하는 항목이므로 합이 아닌 4로 나누어 평균을 구하며, 타 헝목은 4분기 기준 합을 그대로 사용한다.
#2. 계정과 종목코드별 그룹을 나누 후 tail(1) 함수를 통해 가장 최근 데이터만 선택한다.

import numpy as np

sample_fs['ttm'] = np.where(sample_fs['계정'] == '자본',
                            sample_fs['ttm'] / 4, sample_fs['ttm'])
sample_fs = sample_fs.groupby(['계정', '종목코드']).tail(1)

sample_fs.head()

/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/2594106855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_fs['ttm'] = np.where(sample_fs['계정'] == '자본',


,계정,기준일,값,종목코드,공시구분,ttm
3,당기순이익,2024-12-31,77544.0,005930,q,344513.0
7,매출액,2024-12-31,757883.0,005930,q,3008709.0
11,영업활동으로인한현금흐름,2024-12-31,220223.0,005930,q,729826.0
15,자본,2024-12-31,4021920.0,005930,q,964947.5


In [112]:
sample_fs_merge = sample_fs[['계정', '종목코드', 'ttm']].merge(
    ticker_list[['종목코드', '시가총액', '기준일']], on='종목코드')
sample_fs_merge['시가총액'] = sample_fs_merge['시가총액']/100000000

sample_fs_merge.head()

,계정,종목코드,ttm,시가총액,기준일
0,당기순이익,005930,344513.0,3539940.0,2025-03-25
1,매출액,005930,3008709.0,3539940.0,2025-03-25
2,영업활동으로인한현금흐름,005930,729826.0,3539940.0,2025-03-25
3,자본,005930,964947.5,3539940.0,2025-03-25


In [114]:
sample_fs_merge['value'] = sample_fs_merge['시가총액'] / sample_fs_merge['ttm']
sample_fs_merge['지표'] = np.where(
    sample_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        sample_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(sample_fs_merge['계정'] == '자본', 'PBR',
                 np.where(sample_fs_merge['계정'] == '당기순이익', 'PER', None))))

sample_fs_merge

,계정,종목코드,ttm,시가총액,기준일,value,지표
0,당기순이익,005930,344513.0,3539940.0,2025-03-25,10.275200,PER
1,매출액,005930,3008709.0,3539940.0,2025-03-25,1.176564,PSR
2,영업활동으로인한현금흐름,005930,729826.0,3539940.0,2025-03-25,4.850389,PCR
3,자본,005930,964947.5,3539940.0,2025-03-25,3.668531,PBR


In [116]:
#배당수익률
ticker_list_sample = ticker_list[ticker_list['종목코드'] == '005930'].copy()
ticker_list_sample['DY'] = ticker_list_sample['주당배당금'] / ticker_list_sample['종가']

ticker_list_sample.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분,DY
254,005930,삼성전자,KOSPI,59800.0,3.539940e+14,2025-03-25,2131.0,4737.0,52002.0,1444.0,보통주,0.024147


### 전 종목 가치지표 계산

In [ ]:
# 📦 패키지 불러오기
import pymysql
import pandas as pd
import numpy as np

# ✅ pymysql로 DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
cursor = conn.cursor()

# 📌 분기 재무제표 불러오기
query_fs = """
    SELECT * FROM kor_fs
    WHERE 공시구분 = 'q'
      AND 계정 IN ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
"""
kor_fs = pd.read_sql(query_fs, con=conn)

# 📌 티커 리스트 불러오기 (보통주 기준 최신일자)
query_ticker = """
    SELECT * FROM kor_ticker
    WHERE 기준일 = (SELECT MAX(기준일) FROM kor_ticker)
      AND 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query_ticker, con=conn)

# 🔚 연결 종료
cursor.close()
conn.close()

/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/3638052780.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  kor_fs = pd.read_sql(query_fs, con=conn)
/var/folders/y8/w9lwkcq57vz35b4nvfmgcn000000gn/T/ipykernel_22844/3638052780.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ticker_list = pd.read_sql(query_ticker, con=conn)


In [125]:
# TTM 구하기
kor_fs = kor_fs.sort_values(['종목코드', '계정', '기준일'])
kor_fs['ttm'] = kor_fs.groupby(['종목코드', '계정'], as_index=False)['값'].rolling(
    window=4, min_periods=4).sum()['값']

# 자본은 평균 구하기
kor_fs['ttm'] = np.where(kor_fs['계정'] == '자본', kor_fs['ttm'] / 4,
                         kor_fs['ttm'])
kor_fs = kor_fs.groupby(['계정', '종목코드']).tail(1)

In [133]:
# TTM 기준으로 계산된 재무제표 테이블과 티커리스트 테이블을 병합.
kor_fs_merge = kor_fs[['계정', '종목코드',
                       'ttm']].merge(ticker_list[['종목코드', '시가총액', '기준일']],
                                     on='종목코드')
kor_fs_merge['시가총액'] = kor_fs_merge['시가총액'] / 100000000

kor_fs_merge['value'] = kor_fs_merge['시가총액'] / kor_fs_merge['ttm']
kor_fs_merge['value'] = kor_fs_merge['value'].round(4)
kor_fs_merge['지표'] = np.where(
    kor_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        kor_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(kor_fs_merge['계정'] == '자본', 'PBR',
                 np.where(kor_fs_merge['계정'] == '당기순이익', 'PER', None))))

kor_fs_merge.rename(columns={'value': '값'}, inplace=True)
kor_fs_merge = kor_fs_merge[['종목코드', '기준일', '지표', '값']]
kor_fs_merge = kor_fs_merge.replace([np.inf, -np.inf, np.nan], None)

kor_fs_merge.head(4)

,종목코드,기준일,지표,값
0,000020,2025-03-25,PER,81.9324
1,000020,2025-03-25,PSR,0.3701
2,000020,2025-03-25,PCR,-43.0145
3,000020,2025-03-25,PBR,0.4247


In [ ]:
import pymysql

# 새로 DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
mycursor = conn.cursor()

# INSERT 쿼리
query = """
    INSERT INTO kor_value (종목코드, 기준일, 지표, 값)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    값 = VALUES(값)
"""

# 실행
mycursor.executemany(query, args_fs)
conn.commit()

# 연결 종료
mycursor.close()
conn.close()


In [139]:
ticker_list['값'] = ticker_list['주당배당금'] / ticker_list['종가']
ticker_list['값'] = ticker_list['값'].round(4)
ticker_list['지표'] = 'DY'
dy_list = ticker_list[['종목코드', '기준일', '지표', '값']]
dy_list = dy_list.replace([np.inf, -np.inf, np.nan], None)
dy_list = dy_list[dy_list['값'] != 0]

dy_list.head()

,종목코드,기준일,지표,값
0,000020,2025-03-25,DY,0.0292
2,000050,2025-03-25,DY,0.0193
3,000070,2025-03-25,DY,0.0580
4,000080,2025-03-25,DY,0.0481
5,000100,2025-03-25,DY,0.0036


In [ ]:
# 배당수익률 역시 kor_value 테이블에 upsert 방식으로 저장한 후, DB와의 연결을 종료
import pymysql

# ✅ 다시 DB 연결
con = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='badger18',
    passwd='caraway@874Q',
    db='stock_db',
    charset='utf8'
)
mycursor = con.cursor()

# ✅ 실행할 쿼리
query = """
    INSERT INTO kor_value (종목코드, 기준일, 지표, 값)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    값 = VALUES(값)
"""

# ✅ 실행할 데이터
args_dy = dy_list.values.tolist()

# ✅ 실행
mycursor.executemany(query, args_dy)
con.commit()

# ✅ 연결 종료
mycursor.close()
con.close()